<a href="https://colab.research.google.com/github/Abdulbarikassim/FaceEmotionDetection/blob/main/mobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
data_set = '/content/drive/MyDrive/datasets/ckextended.csv'

In [ ]:
import pandas as pd

df = pd.read_csv(data_set)

df.columns

Index(['emotion', 'pixels', 'Usage'], dtype='object')

In [ ]:
!git config --global user.email "abdicsxhaka10@gmail.com"
!git config --global user.name "Abdulbarikassim"

In [ ]:
#implementation of the mobileNet architecture.
from os.path import basename
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator

#parameters

IMG_SIZE = (224,224)
BATCH_SIZE = 32
NUM_CLASS = 7

# Data augementation and preprocessing

augmented_train_data  = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

augemented_test_data = ImageDataGenerator(rescale=1./255)

train_data = augmented_train_data.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_data = augemented_test_data.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

#load the mobileNet2  pre-trained with imagenet, excluding the top layer
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224,224,3))

#freeze the base model layers
base_model.trainable = False

#adding my custom layers on top of the mobileNetV2
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(NUM_CLASS, activation='softmax')(x)

#creating the final model
model = Model(inputs=base_model.input,outputs=predictions)

#compile the model
model.compile(optimizer=Adam(learning_rate=0.0005),loss='categorical_crossentropy',metrics=['accuracy'])


#train the model
trained_model = model.fit(
    train_data,
    steps_per_epoch=train_data.samples // BATCH_SIZE,
    epochs=10,
    validation_data=test_data,
    validation_steps=test_data.samples // BATCH_SIZE
)








Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
897/897 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.3357 - loss: 1.7409

KeyboardInterrupt: 

In [ ]:
#let see the model summary

model.summary()

In [ ]:
#plot the training history
import matplotlib.pyplot as plt

acc = trained_model.history['accuracy']
val_acc = trained_model.history['val_accuracy']
loss = trained_model.history['loss']
val_loss = trained_model.history['val_loss']


epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()
